# Prédiction de l'impact de sanctions économiques

## Notebook 1 - Modèle "naïf" (pas de feature engineering)

In [6]:
# import required libraries

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier

pd.set_option('display.max_columns', 80)
sns.set_style("whitegrid")
%matplotlib inline

np.random.seed(0)

In [25]:
# import the data

base_df = pd.read_csv("input/sanctions.csv.gz", index_col=0)
print(base_df.shape)
# for col, typ in zip(base_df.columns, base_df.dtypes):
#     print(col, "=>", typ)
base_df.head(3)

(204, 38)


,Sender c,Sender 2c,Sender 3c,Targetd,Goal,US casee,US unilateral casef,Foreign policy goal categoryg,First yearh,Last yeari,Policy resultj,Sanctions contributionk,Success \nscorel,Companion policiesm,International cooperationn,International assistanceo,Cooperating international organizationp,International organization senderq,International organization sender & target membersr,Length (years)s,Prior relationst,Regime Type \n(DEMOC 1st year)u,Regime Type2 (Polity2 1st year)v,Regime Type3 \n(Scale 1st year)w,Political stability priorx,Political stability duringy,Cost to targetz,Cost to target (percent of GNP)aa,Cost to target per capitabb,Trade linkagecc,GNP \nratiodd,Health and stabilityee,Sanction typeff,Cost to sendergg,"GDP growth (percent, 5-year average)hh","Inflation (percent,\n 3-year average)ii",Target International Monetary Fund codejj,Country groupkk
Case no.b,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
14-1,UNITED KINGDOM,NaN,NaN,GERMANY,Military victory,0,0,4,1914,1918,4,3,12,R,4,A,--,0,0,4,1,5,2,2,"0,0","0,0",843,"7,1","12,58",9,1,3,"F,X,M",4,"3,6","2,9",134,1
17-1,UNITED STATES,NaN,NaN,JAPAN,Shipping for Allies,1,1,5,1917,1918,2,2,4,--,1,--,--,0,0,1,2,5,1,2,"0,0","0,0",23,"0,8","0,44","20,5",13,3,X,2,"5,4","25,6",158,1
18-1,UNITED KINGDOM,NaN,NaN,RUSSIA,Destabilize Bolsheviks,0,0,2,1918,1920,1,2,2,"R,Q",4,--,--,0,0,2,1,1,-1,2,"0,1","0,0",446,"4,1","2,49","18,5",1,1,"F,X,M",3,n.a.,n.a.,922,2


In [26]:
# Drop information that is unknown at decision time

unknowns = [
    "Last yeari",
    "Length (years)s",
    "Political stability duringy",
    "Cost to targetz",
    "Cost to target (percent of GNP)aa",
    "Cost to target per capitabb",
    "Cost to sendergg",
    "Target International Monetary Fund codejj"
]

df = base_df.drop(unknowns, axis=1)

print(df.shape)
df.head(3)

(204, 30)


,Sender c,Sender 2c,Sender 3c,Targetd,Goal,US casee,US unilateral casef,Foreign policy goal categoryg,First yearh,Policy resultj,Sanctions contributionk,Success \nscorel,Companion policiesm,International cooperationn,International assistanceo,Cooperating international organizationp,International organization senderq,International organization sender & target membersr,Prior relationst,Regime Type \n(DEMOC 1st year)u,Regime Type2 (Polity2 1st year)v,Regime Type3 \n(Scale 1st year)w,Political stability priorx,Trade linkagecc,GNP \nratiodd,Health and stabilityee,Sanction typeff,"GDP growth (percent, 5-year average)hh","Inflation (percent,\n 3-year average)ii",Country groupkk
Case no.b,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
14-1,UNITED KINGDOM,NaN,NaN,GERMANY,Military victory,0,0,4,1914,4,3,12,R,4,A,--,0,0,1,5,2,2,"0,0",9,1,3,"F,X,M","3,6","2,9",1
17-1,UNITED STATES,NaN,NaN,JAPAN,Shipping for Allies,1,1,5,1917,2,2,4,--,1,--,--,0,0,2,5,1,2,"0,0","20,5",13,3,X,"5,4","25,6",1
18-1,UNITED KINGDOM,NaN,NaN,RUSSIA,Destabilize Bolsheviks,0,0,2,1918,1,2,2,"R,Q",4,--,--,0,0,1,1,-1,2,"0,1","18,5",1,1,"F,X,M",n.a.,n.a.,2


In [27]:
df.columns

Index(['Sender c', 'Sender 2c', 'Sender 3c', 'Targetd', 'Goal', 'US casee',
       'US unilateral casef', 'Foreign policy goal categoryg', 'First yearh',
       'Policy resultj', 'Sanctions contributionk', 'Success \nscorel',
       'Companion policiesm', 'International cooperationn',
       'International assistanceo', 'Cooperating international organizationp',
       'International organization senderq',
       'International organization sender & target membersr',
       'Prior relationst', 'Regime Type \n(DEMOC 1st year)u',
       'Regime Type2 (Polity2 1st year)v', 'Regime Type3 \n(Scale 1st year)w',
       'Political stability priorx', 'Trade linkagecc', 'GNP \nratiodd',
       'Health and stabilityee', 'Sanction typeff',
       'GDP growth (percent, 5-year average)hh',
       'Inflation (percent,\n 3-year average)ii', 'Country groupkk'],
      dtype='object')

In [28]:
new_names = [
    'Sender1', 'Sender2', 'Sender3', 'Target', 'Goal', 'US case',
    'US unilateral case', 'Foreign policy goal category', 'First year',
    'Policy result', 'Sanctions contribution', 'Success',
    'Companion policies', 'International cooperation',
    'International assistance', 'Cooperating international organization',
    'International organization sender',
    'International organization sender & target members',
    'Prior relationst', 'Regime Type',
    'Regime Type2', 'Regime Type3',
    'Political stability prior', 'Trade linkage', 'GNP \nratio',
    'Health and stability', 'Sanction type',
    'GDP growth',
    'Inflation', 'Country group']

In [29]:
name_dict = dict(zip(df.columns, new_names))
name_dict

{'Sender c': 'Sender1',
 'Sender 2c': 'Sender2',
 'Sender 3c': 'Sender3',
 'Targetd': 'Target',
 'Goal': 'Goal',
 'US casee': 'US case',
 'US unilateral casef': 'US unilateral case',
 'Foreign policy goal categoryg': 'Foreign policy goal category',
 'First yearh': 'First year',
 'Policy resultj': 'Policy result',
 'Sanctions contributionk': 'Sanctions contribution',
 'Success \nscorel': 'Success',
 'Companion policiesm': 'Companion policies',
 'International cooperationn': 'International cooperation',
 'International assistanceo': 'International assistance',
 'Cooperating international organizationp': 'Cooperating international organization',
 'International organization senderq': 'International organization sender',
 'International organization sender & target membersr': 'International organization sender & target members',
 'Prior relationst': 'Prior relationst',
 'Regime Type \n(DEMOC 1st year)u': 'Regime Type',
 'Regime Type2 (Polity2 1st year)v': 'Regime Type2',
 'Regime Type3 \n(

In [30]:
df = df.rename(columns=name_dict)

In [31]:
df["score"] = (df['Success'] > 9).astype(int)

In [32]:
df.columns

Index(['Sender1', 'Sender2', 'Sender3', 'Target', 'Goal', 'US case',
       'US unilateral case', 'Foreign policy goal category', 'First year',
       'Policy result', 'Sanctions contribution', 'Success',
       'Companion policies', 'International cooperation',
       'International assistance', 'Cooperating international organization',
       'International organization sender',
       'International organization sender & target members',
       'Prior relationst', 'Regime Type', 'Regime Type2', 'Regime Type3',
       'Political stability prior', 'Trade linkage', 'GNP \nratio',
       'Health and stability', 'Sanction type', 'GDP growth', 'Inflation',
       'Country group', 'score'],
      dtype='object')

In [34]:
df = df[['Sender1', 'Sender2', 'Sender3', 'Target', 'Goal', 'US case',
       'US unilateral case', 'Foreign policy goal category', 'First year',
       'Policy result', 'Sanctions contribution', 'Success', 'score',
       'Companion policies', 'International cooperation',
       'International assistance', 'Cooperating international organization',
       'International organization sender',
       'International organization sender & target members',
       'Prior relationst', 'Regime Type', 'Regime Type2', 'Regime Type3',
       'Political stability prior', 'Trade linkage', 'GNP \nratio',
       'Health and stability', 'Sanction type', 'GDP growth', 'Inflation',
       'Country group']]

In [35]:
df

,Sender1,Sender2,Sender3,Target,Goal,US case,US unilateral case,Foreign policy goal category,First year,Policy result,Sanctions contribution,Success,score,Companion policies,International cooperation,International assistance,Cooperating international organization,International organization sender,International organization sender & target members,Prior relationst,Regime Type,Regime Type2,Regime Type3,Political stability prior,Trade linkage,GNP \nratio,Health and stability,Sanction type,GDP growth,Inflation,Country group
Case no.b,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
14-1,UNITED KINGDOM,NaN,NaN,GERMANY,Military victory,0,0,4,1914,4,3,12,1,R,4,A,--,0,0,1,5,2,2,"0,0",9,1,3,"F,X,M","3,6","2,9",1
17-1,UNITED STATES,NaN,NaN,JAPAN,Shipping for Allies,1,1,5,1917,2,2,4,0,--,1,--,--,0,0,2,5,1,2,"0,0","20,5",13,3,X,"5,4","25,6",1
18-1,UNITED KINGDOM,NaN,NaN,RUSSIA,Destabilize Bolsheviks,0,0,2,1918,1,2,2,0,"R,Q",4,--,--,0,0,1,1,-1,2,"0,1","18,5",1,1,"F,X,M",n.a.,n.a.,2
21-1,LEAGUE OF NATIONS,NaN,NaN,YUGOSLAVIA,Military disruption vs. Albania,0,0,3,1921,4,4,16,1,--,4,--,--,1,1,2,3,0,2,"0,4","26,5",37,2,--,"-1,5",n.a.,2
25-1,LEAGUE OF NATIONS,NaN,NaN,GREECE,Withdraw from Bulgaria,0,0,3,1925,4,4,16,1,--,4,--,--,1,1,2,0,-6,1,"0,9",36,56,2,--,"3,3","48,8",1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
98-3,TURKEY,NaN,NaN,ITALY,Kurdish leader,0,0,1,1998,3,3,9,0,--,1,--,--,0,0,3,10,10,3,"0,0","1,4","0,16",3,--,"1,5","3,7",1
99-1,UNITED NATIONS,UNITED STATES,NaN,AFGHANISTAN,Extradite Osama bin Laden,1,0,1,1999,1,2,2,0,R,4,--,--,1,1,1,0,-7,1,"0,5",100,5939,1,"F,X,M","8,2",n.a.,5
99-2,UNITED STATES,EUROPEAN UNION,FRANCE,IVORY COAST,"Coup, democracy",1,0,2,1999,2,3,6,0,--,2,--,"ECOWAS, OAU",1,1,3,-88,-1,2,"0,1",54,1408,1,F,"5,2","3,7",6


In [37]:
df_scored = df[['Sender1', 'Sender2', 'Sender3', 'Target', 'Goal', 'US case',
       'US unilateral case', 'Foreign policy goal category', 'First year', 'score',
       'Companion policies', 'International cooperation',
       'International assistance', 'Cooperating international organization',
       'International organization sender',
       'International organization sender & target members',
       'Prior relationst', 'Regime Type', 'Regime Type2', 'Regime Type3',
       'Political stability prior', 'Trade linkage', 'GNP \nratio',
       'Health and stability', 'Sanction type', 'GDP growth', 'Inflation',
       'Country group']]